In [4]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np
from fastapi import FastAPI, Query
from pydantic import BaseModel
from typing import List
import joblib
import re

In [5]:
# Ruta al archivo Parquet
ruta_archivo_parquet_sitios = r'C:\Users\guard\OneDrive\Desktop\Henry Data Science\Proyecto-FInal\Datos\gm_sitios_NJNY_20250305.parquet'

# Leer el archivo Parquet en un DataFrame
df = pd.read_parquet(ruta_archivo_parquet_sitios)

# Mostrar las columnas disponibles
print("Columnas disponibles en el DataFrame:")
print(df.columns)

# Mostrar las primeras filas del DataFrame para tener una idea del contenido
print("\nPrimeras filas del DataFrame:")
df.head()

Columnas disponibles en el DataFrame:
Index(['gmap_id', 'name', 'street_address', 'city', 'state', 'zip_code',
       'latitude', 'longitude', 'avg_rating', 'num_of_reviews', 'price',
       'Monday_open', 'Monday_close', 'Tuesday_open', 'Tuesday_close',
       'Wednesday_open', 'Wednesday_close', 'Thursday_open', 'Thursday_close',
       'Friday_open', 'Friday_close', 'Saturday_open', 'Saturday_close',
       'Sunday_open', 'Sunday_close', 'Delivery', 'Dine-in', 'Takeout',
       'Good for kids', 'Casual', 'Dinner', 'Lunch'],
      dtype='object')

Primeras filas del DataFrame:


,gmap_id,name,street_address,city,state,zip_code,latitude,longitude,avg_rating,num_of_reviews,...,Saturday_close,Sunday_open,Sunday_close,Delivery,Dine-in,Takeout,Good for kids,Casual,Dinner,Lunch
index,,,,,,,,,,,,,,,,,,,,,
0,0x4ccab4b3af9c7c85:0x9defa19fe4e295c,Alpine Pizza,1104 U.S. 9,Schroon Lake,NY,12870,43.837433,-73.761254,4.3,45,...,21,16,21,1,1,1,1,1,1,0
26,0x89de5f9627d4e973:0x5b0cbc68d57ffefb,Stewart's Shops,4192 NY-30,Amsterdam,NY,12010,43.017033,-74.194120,2.2,15,...,0,4,0,1,0,0,0,0,0,0
34,0x89def3a6bb7c4091:0xde094955651ec4bf,Al's Pizzeria,201 N Main St,Gloversville,NY,12078,43.056137,-74.342115,4.2,28,...,21,12,21,0,0,1,1,1,0,0
53,0x89c3acab12c10d5d:0x73285fc4bd781796,Gulistan Pizza,783 S Orange Ave,Newark,NJ,07106,40.745207,-74.220116,3.3,8,...,None,None,None,1,0,0,1,0,0,0
59,0x89c259f744b20deb:0xb536eb5ee402f92f,FREEHOLD In The Park,20 Union Square W,New York,NY,10003,40.736450,-73.989926,4.2,78,...,0,12,0,1,1,1,1,1,0,0


In [14]:
# Seleccionar características relevantes
caracteristicas_relevantes = ['avg_rating', 'num_of_reviews', 'price','Monday_open', 'Monday_close', 'Tuesday_open', 'Tuesday_close', 'Wednesday_open', 'Wednesday_close', 'Thursday_open', 'Thursday_close', 'Friday_open', 'Friday_close', 'Saturday_open', 'Saturday_close', 'Sunday_open', 'Sunday_close', 'Delivery', 'Dine-in', 'Takeout', 'Good for kids', 'Casual', 'Dinner', 'Lunch']

# Crear un nuevo DataFrame con las características seleccionadas
df_relevantes = df[caracteristicas_relevantes]

# Mostrar las primeras filas del nuevo DataFrame
df_relevantes.head()

,avg_rating,num_of_reviews,price,Monday_open,Monday_close,Tuesday_open,Tuesday_close,Wednesday_open,Wednesday_close,Thursday_open,...,Saturday_close,Sunday_open,Sunday_close,Delivery,Dine-in,Takeout,Good for kids,Casual,Dinner,Lunch
index,,,,,,,,,,,,,,,,,,,,,
0,4.3,45,$$,None,None,None,None,16,21,16,...,21,16,21,1,1,1,1,1,1,0
26,2.2,15,$$,4,0,4,0,4,0,4,...,0,4,0,1,0,0,0,0,0,0
34,4.2,28,None,12,21,12,21,12,21,12,...,21,12,21,0,0,1,1,1,0,0
53,3.3,8,None,None,None,None,None,None,None,None,...,None,None,None,1,0,0,1,0,0,0
59,4.2,78,None,12,0,12,0,12,0,12,...,0,12,0,1,1,1,1,1,0,0


In [15]:
# Convertir las columnas de horarios a formato numérico
horarios_columnas = ['Monday_open', 'Monday_close', 'Tuesday_open', 'Tuesday_close', 'Wednesday_open', 'Wednesday_close', 'Thursday_open', 'Thursday_close', 'Friday_open', 'Friday_close', 'Saturday_open', 'Saturday_close', 'Sunday_open', 'Sunday_close']

for col in horarios_columnas:
    df_relevantes[col] = pd.to_numeric(df_relevantes[col], errors='coerce')

# Mostrar las primeras filas del DataFrame para verificar la conversión
df_relevantes.head()

C:\Users\guard\AppData\Local\Temp\ipykernel_19968\100129164.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevantes[col] = pd.to_numeric(df_relevantes[col], errors='coerce')


,avg_rating,num_of_reviews,price,Monday_open,Monday_close,Tuesday_open,Tuesday_close,Wednesday_open,Wednesday_close,Thursday_open,...,Saturday_close,Sunday_open,Sunday_close,Delivery,Dine-in,Takeout,Good for kids,Casual,Dinner,Lunch
index,,,,,,,,,,,,,,,,,,,,,
0,4.3,45,$$,NaN,NaN,NaN,NaN,16.0,21.0,16.0,...,21.0,16.0,21.0,1,1,1,1,1,1,0
26,2.2,15,$$,4.0,0.0,4.0,0.0,4.0,0.0,4.0,...,0.0,4.0,0.0,1,0,0,0,0,0,0
34,4.2,28,None,12.0,21.0,12.0,21.0,12.0,21.0,12.0,...,21.0,12.0,21.0,0,0,1,1,1,0,0
53,3.3,8,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,0,0,1,0,0,0
59,4.2,78,None,12.0,0.0,12.0,0.0,12.0,0.0,12.0,...,0.0,12.0,0.0,1,1,1,1,1,0,0


In [16]:
# Eliminar filas con valores NaN
df_relevantes = df_relevantes.dropna()

# Mostrar las primeras filas del DataFrame para verificar la eliminación
df_relevantes.head()

,avg_rating,num_of_reviews,price,Monday_open,Monday_close,Tuesday_open,Tuesday_close,Wednesday_open,Wednesday_close,Thursday_open,...,Saturday_close,Sunday_open,Sunday_close,Delivery,Dine-in,Takeout,Good for kids,Casual,Dinner,Lunch
index,,,,,,,,,,,,,,,,,,,,,
26,2.2,15,$$,4.0,0.0,4.0,0.0,4.0,0.0,4.0,...,0.0,4.0,0.0,1,0,0,0,0,0,0
113,4.1,28,$,11.0,21.0,11.0,21.0,11.0,21.0,11.0,...,21.0,11.0,21.0,1,1,1,1,1,1,1
207,4.3,54,$,10.0,23.0,10.0,23.0,10.0,23.0,10.0,...,23.0,11.0,23.0,1,0,0,1,0,0,0
301,3.8,18,$$,5.0,23.0,5.0,23.0,5.0,23.0,5.0,...,23.0,5.0,23.0,1,0,0,0,0,0,0
462,4.3,75,$,11.0,21.0,11.0,21.0,11.0,21.0,11.0,...,21.0,11.0,20.0,1,1,1,1,1,1,1


In [19]:
# Reemplazar los símbolos de dólar en la columna 'price' por valores numéricos
df_relevantes['price'] = df_relevantes['price'].replace({'$': 1, '$$': 2, '$$$': 3, '$$$$': 4, '$$$$$': 5})

# Mostrar las primeras filas del DataFrame para verificar la sustitución
df_relevantes.head()

C:\Users\guard\AppData\Local\Temp\ipykernel_19968\2079337929.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_relevantes['price'] = df_relevantes['price'].replace({'$': 1, '$$': 2, '$$$': 3, '$$$$': 4, '$$$$$': 5})


,avg_rating,num_of_reviews,price,Monday_open,Monday_close,Tuesday_open,Tuesday_close,Wednesday_open,Wednesday_close,Thursday_open,...,Saturday_close,Sunday_open,Sunday_close,Delivery,Dine-in,Takeout,Good for kids,Casual,Dinner,Lunch
index,,,,,,,,,,,,,,,,,,,,,
26,2.2,15,2,4.0,0.0,4.0,0.0,4.0,0.0,4.0,...,0.0,4.0,0.0,1,0,0,0,0,0,0
113,4.1,28,1,11.0,21.0,11.0,21.0,11.0,21.0,11.0,...,21.0,11.0,21.0,1,1,1,1,1,1,1
207,4.3,54,1,10.0,23.0,10.0,23.0,10.0,23.0,10.0,...,23.0,11.0,23.0,1,0,0,1,0,0,0
301,3.8,18,2,5.0,23.0,5.0,23.0,5.0,23.0,5.0,...,23.0,5.0,23.0,1,0,0,0,0,0,0
462,4.3,75,1,11.0,21.0,11.0,21.0,11.0,21.0,11.0,...,21.0,11.0,20.0,1,1,1,1,1,1,1


In [20]:
df_relevantes.to_csv('data_preprocesada.csv', index=False)

In [35]:
# Seleccionar las columnas a agregar
columnas_a_agregar = ['name', 'street_address', 'zip_code']

# Agregar las columnas al inicio del dataframe df_relevantes
df_recomendacion = df[columnas_a_agregar].join(df_relevantes)
df_recomendacion = df_recomendacion.dropna()

# Guardar el nuevo dataframe en un archivo CSV
df_recomendacion.to_csv('data_recomendacion.csv', index=False)



In [21]:
# Definir el número de vecinos
n_vecinos = 5

# Crear una instancia del modelo NearestNeighbors
modelo_knn = NearestNeighbors(n_neighbors=n_vecinos)

# Entrenar el modelo con los datos relevantes
modelo_knn.fit(df_relevantes)

print("Modelo NearestNeighbors entrenado con éxito.")

Modelo NearestNeighbors entrenado con éxito.


In [22]:
import joblib

# Guardar el modelo entrenado en un archivo
ruta_modelo = 'modelo_knn.pkl'
joblib.dump(modelo_knn, ruta_modelo)

print(f"Modelo guardado en {ruta_modelo}")

Modelo guardado en modelo_knn.pkl


In [23]:
def recomendar_restaurantes(zip_code: str):
    # Filtrar los restaurantes por el código postal
    df_filtrado = df[df['zip_code'] == zip_code]
    
    # Seleccionar los 10 restaurantes con mayor número de comentarios
    top_10_reviews = df_filtrado.nlargest(10, 'num_of_reviews')
    
    # Seleccionar los 5 restaurantes con mayor puntuación promedio
    top_5_rating = top_10_reviews.nlargest(5, 'avg_rating')
    
    # Mostrar la información de los restaurantes recomendados
    for index, row in top_5_rating.iterrows():
        print(f"El restaurante '{row['name']}', ubicado en '{row['street_address']}', posee {row['num_of_reviews']} comentarios, y el promedio de su puntuación es {row['avg_rating']}.")

# Ejemplo de uso
recomendar_restaurantes('10003')

El restaurante 'Bruno Pizza', ubicado en '204 E 13th St', posee 176 comentarios, y el promedio de su puntuación es 4.5.
El restaurante 'Atlas Cafe', ubicado en '73 2nd Ave', posee 124 comentarios, y el promedio de su puntuación es 4.5.
El restaurante 'DIA', ubicado en '58 2nd Ave', posee 46 comentarios, y el promedio de su puntuación es 4.5.
El restaurante 'Tarallucci e Vino', ubicado en '163 1st Avenue', posee 218 comentarios, y el promedio de su puntuación es 4.4.
El restaurante 'Old Fashion Pizzeria', ubicado en '244 E 13th St', posee 58 comentarios, y el promedio de su puntuación es 4.4.


In [32]:
def recomendar_restaurantes(zip_code: str, dia: str, hora: float):
    # Filtrar los restaurantes por el código postal
    df_filtrado = df[df['zip_code'] == zip_code]
    
    # Convertir las columnas de horarios a formato numérico
    df_filtrado[f'{dia}_open'] = pd.to_numeric(df_filtrado[f'{dia}_open'], errors='coerce')
    df_filtrado[f'{dia}_close'] = pd.to_numeric(df_filtrado[f'{dia}_close'], errors='coerce')
    
    # Filtrar los restaurantes por la hora de apertura y cierre
    df_filtrado = df_filtrado[(df_filtrado[f'{dia}_open'] <= hora) & (df_filtrado[f'{dia}_close'] >= hora)]
    
    # Seleccionar los 10 restaurantes con mayor número de comentarios
    top_10_reviews = df_filtrado.nlargest(10, 'num_of_reviews')
    
    # Seleccionar los 5 restaurantes con mayor puntuación promedio
    top_5_rating = top_10_reviews.nlargest(5, 'avg_rating')
    
    # Mostrar la información de los restaurantes recomendados
    for index, row in top_5_rating.iterrows():
        print(f"El restaurante '{row['name']}', ubicado en '{row['street_address']}', posee {row['num_of_reviews']} comentarios, y el promedio de su puntuación es {row['avg_rating']}.")

# Ejemplo de uso
recomendar_restaurantes('10003', 'Tuesday', 16.0)

El restaurante 'Tarallucci e Vino', ubicado en '163 1st Avenue', posee 218 comentarios, y el promedio de su puntuación es 4.4.
El restaurante 'Old Fashion Pizzeria', ubicado en '244 E 13th St', posee 58 comentarios, y el promedio de su puntuación es 4.4.
El restaurante 'Mandolino', ubicado en '137 E 13th Street Condominium, 137 E 13th St', posee 58 comentarios, y el promedio de su puntuación es 4.4.


C:\Users\guard\AppData\Local\Temp\ipykernel_19968\1453446445.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado[f'{dia}_open'] = pd.to_numeric(df_filtrado[f'{dia}_open'], errors='coerce')
C:\Users\guard\AppData\Local\Temp\ipykernel_19968\1453446445.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado[f'{dia}_close'] = pd.to_numeric(df_filtrado[f'{dia}_close'], errors='coerce')


In [38]:
pd.options.mode.chained_assignment = None  # Desactivar los warnings de asignación en cadena

def recomendar_restaurantes(zip_code: str, dia: str, hora: float):
    # Filtrar los restaurantes por el código postal
    df_filtrado = df[df['zip_code'] == zip_code]
    
    # Convertir las columnas de horarios a formato numérico
    df_filtrado[f'{dia}_open'] = pd.to_numeric(df_filtrado[f'{dia}_open'], errors='coerce')
    df_filtrado[f'{dia}_close'] = pd.to_numeric(df_filtrado[f'{dia}_close'], errors='coerce')
    
    # Filtrar los restaurantes por la hora de apertura y cierre
    df_filtrado = df_filtrado[(df_filtrado[f'{dia}_open'] <= hora) & (df_filtrado[f'{dia}_close'] >= hora)]
    
    # Seleccionar los 10 restaurantes con mayor número de comentarios
    top_10_reviews = df_filtrado.nlargest(10, 'num_of_reviews')
    
    # Seleccionar los 5 restaurantes con mayor puntuación promedio
    top_5_rating = top_10_reviews.nlargest(5, 'avg_rating')
    
    # Mostrar la información de los restaurantes recomendados
    for index, row in top_5_rating.iterrows():
        print(f"El restaurante '{row['name']}', ubicado en '{row['street_address']}', posee {row['num_of_reviews']} comentarios, y el promedio de su puntuación es {row['avg_rating']}.")

# Ejemplo de uso
recomendar_restaurantes('11220', 'Tuesday', 16.0)

El restaurante 'Charles Pizzeria', ubicado en '4910 5th Ave', posee 218 comentarios, y el promedio de su puntuación es 4.5.
El restaurante 'Viiza', ubicado en '5401 8th Ave', posee 33 comentarios, y el promedio de su puntuación es 4.2.
El restaurante 'Rax's Pizza', ubicado en '4613 5th Ave', posee 28 comentarios, y el promedio de su puntuación es 4.1.
El restaurante 'Tony's Pizza', ubicado en '6112 4th Ave', posee 28 comentarios, y el promedio de su puntuación es 3.9.
